In [30]:
import itertools
import random 
import copy

In [31]:
class Species:
    def __init__(self, cal_needed: float, cal_provided: float):
        self.cal_needed = cal_needed
        self.cal_provided = cal_provided

class Animal(Species):
    def __init__(self, animal_id: int, cal_needed: float, cal_provided: float, food_source_ids: list):
        super().__init__(cal_needed, cal_provided)
        self.id = animal_id
        self.food_source_ids = food_source_ids  # IDs of other animals

    # Method to find the actual animal objects from their IDs
    def find_food_sources(self, all_species):
        return [species for species in all_species if species.id in self.food_source_ids]
    
    def eat_from_best_food_source(self,all_species) :
        food_sources = self.find_food_sources(all_species)
        if len(food_sources) ==0 :
            # raise ValueError(f"Not a food chain : this animal can't eat, id : {self.id}")
            return
        food_sources.sort(key=lambda x: x.cal_provided, reverse=True)
        max_cal_provided = food_sources[0].cal_provided
        best_food_sources = [source for source in food_sources if source.cal_provided == max_cal_provided]
        K = len(best_food_sources)

        for source in best_food_sources : 
            cal_consumed = min(self.cal_needed, source.cal_provided)
            if self.cal_needed >= source.cal_provided :
                return
            source.cal_provided -= self.cal_needed / K 
            self.cal_needed -= self.cal_needed / K 

    def display(self): 
        print('Animal ID:', self.id, 'cal_needed:', self.cal_needed, 'cal_provided:', self.cal_provided, 'food_source_ids:', self.food_source_ids)

class Plant(Species):
    def __init__(self, plant_id: int, cal_provided: float, eaters: list):
        super().__init__(0, cal_provided)  # Plants have cal_needed = 0
        self.id = plant_id
        self.eaters = eaters  # IDs of animals that can eat the plant

    # Method to find the actual animal objects from their IDs
    def find_eaters(self, all_species):
        return [species for species in all_species if species.id in self.eaters]

    def display(self): 
        print('Plant ID:', self.id, 'cal_provided:', self.cal_provided, 'eaters:', self.eaters)

In [32]:
# # Creating a list of Random Animal objects
# def create_dataset(N=10) :
#     dataset_animals = []
#     for i in range(N):
#         a = Animal(i, random.randint(3000, 4000), random.randint(3500, 5000), [])
#         dataset_animals.append(a)
#     # Assign random food sources to each animal
#     for animal in dataset_animals:
#         # Choose a random number of food sources (between 1 and 5)
#         num_food_sources = random.randint(1, 5)
#         # Choose random animals from the dataset_animals list as food sources
#         food_sources = random.sample(dataset_animals, num_food_sources)
#         # Update the food_sources attribute of the current animal
#         animal.food_sources = food_sources
#     return dataset_animals

# random_dataset = create_dataset()

In [33]:
def run_food_chain(dataset_species) : 
    dataset_copy = [copy.deepcopy(species) for species in dataset_species]
    dataset_copy = sorted(dataset_copy, key=lambda species: species.cal_provided, reverse=True)
    for species in dataset_copy : 
        sorted_species = sorted(dataset_copy, key=lambda species: species.cal_provided, reverse=True)
        if isinstance(species,Animal) : 
            species.eat_from_best_food_source(sorted_species)
    
    sorted_species = sorted(sorted_species, key=lambda species: species.id)
    return sorted_species

In [34]:
# # Example usage:
# my_dataset = [
#     Animal(0, 4000, 3500, [5, 6]),
#     Animal(1, 3300, 4000, [0]),
#     Animal(2, 4500, 3900, [6, 7]),
#     Animal(3, 3800, 3700, [2, 4]),
#     Animal(4, 3200, 2000, [7]),

#     Plant(5, 5500, []),
#     Plant(6, 5500, []),
#     Plant(7, 4000, []),

#     Animal(8,8500,1800,[0,1,2,3,4,5,6,7]) #apex
# ]

In [35]:
# Example usage:
my_dataset = [
    Plant(1, 5250, []),
    Plant(2, 4600, []),
    Plant(3, 4950, []),

    Animal(4, 4900, 3750, [1,2,3]),
    Animal(5, 3000, 900, [2]),
    Animal(6, 4800, 2500, [3]),
    Animal(7, 3000, 1650, [6]),
    Animal(8, 3150, 3950, [6,4,5]),

    Animal(9, 800, 3400, [8]),
    Animal(10, 5900, 2050, [4,12]),
    Animal(11, 1150, 2150, [8]),
    Animal(12, 4450, 800, [3,2]),
    Animal(13, 2500, 1300, [10,6,12]),

]

In [36]:
# Result at the end of the food chain 
rez = run_food_chain(my_dataset)
for a in rez : 
    a.display()

Plant ID: 1 cal_provided: 350.0 eaters: []
Plant ID: 2 cal_provided: 1600.0 eaters: []
Plant ID: 3 cal_provided: 150.0 eaters: []
Animal ID: 4 cal_needed: 0.0 cal_provided: 600.0 food_source_ids: [1, 2, 3]
Animal ID: 5 cal_needed: 0.0 cal_provided: 900 food_source_ids: [2]
Animal ID: 6 cal_needed: 0.0 cal_provided: 2500 food_source_ids: [3]
Animal ID: 7 cal_needed: 3000 cal_provided: 1650 food_source_ids: [6]
Animal ID: 8 cal_needed: 0.0 cal_provided: 2000.0 food_source_ids: [6, 4, 5]
Animal ID: 9 cal_needed: 0.0 cal_provided: 3400 food_source_ids: [8]
Animal ID: 10 cal_needed: 5900 cal_provided: 2050 food_source_ids: [4, 12]
Animal ID: 11 cal_needed: 0.0 cal_provided: 2150 food_source_ids: [8]
Animal ID: 12 cal_needed: 4450 cal_provided: 800 food_source_ids: [3, 2]
Animal ID: 13 cal_needed: 2500 cal_provided: 1300 food_source_ids: [10, 6, 12]


In [37]:
for a in my_dataset : 
    a.display()
# We keep the inital state of our dataset intact

Plant ID: 1 cal_provided: 5250 eaters: []
Plant ID: 2 cal_provided: 4600 eaters: []
Plant ID: 3 cal_provided: 4950 eaters: []
Animal ID: 4 cal_needed: 4900 cal_provided: 3750 food_source_ids: [1, 2, 3]
Animal ID: 5 cal_needed: 3000 cal_provided: 900 food_source_ids: [2]
Animal ID: 6 cal_needed: 4800 cal_provided: 2500 food_source_ids: [3]
Animal ID: 7 cal_needed: 3000 cal_provided: 1650 food_source_ids: [6]
Animal ID: 8 cal_needed: 3150 cal_provided: 3950 food_source_ids: [6, 4, 5]
Animal ID: 9 cal_needed: 800 cal_provided: 3400 food_source_ids: [8]
Animal ID: 10 cal_needed: 5900 cal_provided: 2050 food_source_ids: [4, 12]
Animal ID: 11 cal_needed: 1150 cal_provided: 2150 food_source_ids: [8]
Animal ID: 12 cal_needed: 4450 cal_provided: 800 food_source_ids: [3, 2]
Animal ID: 13 cal_needed: 2500 cal_provided: 1300 food_source_ids: [10, 6, 12]


In [38]:
def test_subsets(dataset_species, K):
    valid_subsets = []
    subsets = itertools.combinations(dataset_species, K)
    for subset in subsets:
        # Test the food chain for the current subset
        species_after_food_chain = run_food_chain(list(subset))
        # Check if all animals at the end have cal_needed > 0
        valid = True
        for species in species_after_food_chain:
            if species.cal_provided <=0 :
                valid = False
            if isinstance(species, Animal):
                if species.cal_needed != 0:
                    valid = False
                    break
        if valid:
            valid_subsets.append((subset, species_after_food_chain))
    return valid_subsets

In [39]:
valid_subsets = test_subsets(my_dataset, K=8)
for initial_subset_state, final_subset_state in valid_subsets:
    print("Valid subset:")
    print("Initial state:")
    for species in initial_subset_state:
        species.display()
    print("Final state:")
    for species in final_subset_state:
        species.display()
    print()

Valid subset:
Initial state:
Plant ID: 1 cal_provided: 5250 eaters: []
Plant ID: 2 cal_provided: 4600 eaters: []
Plant ID: 3 cal_provided: 4950 eaters: []
Animal ID: 4 cal_needed: 4900 cal_provided: 3750 food_source_ids: [1, 2, 3]
Animal ID: 5 cal_needed: 3000 cal_provided: 900 food_source_ids: [2]
Animal ID: 6 cal_needed: 4800 cal_provided: 2500 food_source_ids: [3]
Animal ID: 8 cal_needed: 3150 cal_provided: 3950 food_source_ids: [6, 4, 5]
Animal ID: 9 cal_needed: 800 cal_provided: 3400 food_source_ids: [8]
Final state:
Plant ID: 1 cal_provided: 350.0 eaters: []
Plant ID: 2 cal_provided: 1600.0 eaters: []
Plant ID: 3 cal_provided: 150.0 eaters: []
Animal ID: 4 cal_needed: 0.0 cal_provided: 600.0 food_source_ids: [1, 2, 3]
Animal ID: 5 cal_needed: 0.0 cal_provided: 900 food_source_ids: [2]
Animal ID: 6 cal_needed: 0.0 cal_provided: 2500 food_source_ids: [3]
Animal ID: 8 cal_needed: 0.0 cal_provided: 3150.0 food_source_ids: [6, 4, 5]
Animal ID: 9 cal_needed: 0.0 cal_provided: 3400 food

- Puffball
- Silver Dollar Fern
- Velvet Red coleus
- Forest cottontail
- Lowland Paca
- Pygmy Brocket
- Margay
- Brown Four-eyed Opossum
- Black eagel
- Coyote
- Peregrine falcon
- Red rumped agouti
- swainson's hawk

In [40]:
# Example usage:
my_dataset = [
    Plant(1, 5250, []),
    Plant(2, 4600, []),
    Plant(3, 4950, []),

    Animal(4, 4900, 3750, [1,2,3]),
    Animal(5, 3000, 900, [2]),
    Animal(6, 4800, 2500, [3]),
    Animal(7, 3000, 1650, [6]),
    Animal(8, 3150, 3950, [6,4,5]),

    Animal(9, 800, 3400, [8]),
    Animal(10, 5900, 2050, [4,12]),
    Animal(11, 1150, 2150, [8]),
    Animal(12, 4450, 800, [3,2]),
    Animal(13, 2500, 1300, [10,6,12]),

]